# Exploring

### We will parse the wikipedia page and create a datafram from those data

In [194]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [195]:
# Parse the wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
soup = BeautifulSoup(requests.get(url).text, "html.parser")
table = soup.find("table")
# Create dataframe
df = pd.read_html(str(table))[0]
# Rename columns
df.columns = ['Postalcode', 'Borough', 'Neighborhood']
# Replace Not assigned by Borough
df.Borough.replace("Not assigned", df.Neighborhood, inplace=True)
# Remove Not Assigned Neighborhoods
df = df.drop(df[(df.Borough == "Not assigned")].index)
# Merge identical Postal Codes
df = df.groupby(['Postalcode','Borough'], sort=False).agg(', '.join)
df.reset_index(inplace=True)

df.shape

(103, 3)

# Adding coordinates

In [196]:
!pip install geocoder
import geocoder # import geocoder

In [197]:
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [198]:
# Get the geo code csv
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
# Rename column postal code
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
# Merge the 2 dataframes
df = pd.merge(df, geo_df, on='Postalcode')
df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Clustering

In [199]:
CLIENT_ID = 'C5P10I52RGGI1IVOHJGQH0YYVDHATZPIOVIFKULYJOA5TDKQ' # your Foursquare ID
CLIENT_SECRET = 'XDCZUMPWEFBD4OMVKCEGF4XWQTAUOXUWPKARU2VRH3YEQVDG' # your Foursquare Secret
VERSION = '20180604'

In [200]:
df_toronto = df[df['Borough'].str.contains('Toronto',regex=False)]
df_toronto

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [201]:
!pip install folium
import folium
from sklearn.cluster import KMeans
import numpy as np 
import matplotlib.cm as cm
import matplotlib.colors as colors

In [202]:
# Visualizing

map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [203]:
k=6
toronto_clustering = df_toronto.drop(['Postalcode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df_toronto.insert(0, 'Cluster Labels', kmeans.labels_)

In [204]:
# Visualizing after KMeans
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood'], df_toronto['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters